In [10]:
import numpy as np
from gwbench import snr
from gwbench import waveform as wfc
from gwbench import injections
from gwbench import network
from gwbench import basic_relations as br
from gwbench import detector_response_derivatives as drds

from pycbc import conversions as conv
from  pycbc.filter.matchedfilter import optimized_match

from astropy.cosmology import Planck18, z_at_value

import gwbench_network_funcs as gwnet

from pycbc.types import FrequencySeries
from pycbc.filter import match
from pycbc.filter import matchedfilter

from pycbc import types, fft, waveform

In [3]:
input_file = 'smooth_powerlaw_pop.npz'
with np.load(input_file, mmap_mode='r') as binaries:
    Mcs = np.array(binaries['Mcs'])
    etas = np.array(binaries['etas'])
    chi1z = np.array(binaries['chi1z'])
    chi2z = np.array(binaries['chi2z'])
    DLs = np.array(binaries['DLs'])
    iotas = np.array(binaries['iotas'])
    ras = np.array(binaries['ras'])
    decs = np.array(binaries['decs'])
    psis = np.array(binaries['psis'])

m1s = conv.mass1_from_mchirp_eta(Mcs, etas)
mtots = conv.mtotal_from_mchirp_eta(Mcs, etas)

# Generate waveforms and derivatives

In [43]:
i = 54837

inj_params = {
    'Mc':    Mcs[i],
    'eta':   etas[i],
    'chi1x': 0.,
    'chi2x': 0.,
    'chi1y': 0.,
    'chi2y': 0.,
    'chi1z': chi1z[i],
    'chi2z': chi2z[i],
    'DL':    DLs[i],
    'tc':    0,
    'phic':  0,
    'iota':  iotas[i],
    'ra':    ras[i],
    'dec':   decs[i],
    'psi':   psis[i],
    'gmst0': 0,
    'hybr': 0.0
} 

net_key = '3G'

approx1 = 'IMRPhenomXAS'
approx2 = 'IMRPhenomD'

d_f = 2**-4
f_low = 5.0
mtotal = conv.mtotal_from_mchirp_eta(inj_params['Mc'], inj_params['eta'])
f_high = np.round(4*br.f_isco_Msolar(mtotal))

net_ap = gwnet.get_hybrid_network_response(inj_params=inj_params, network_key=net_key, f_max=f_high,
                            approximant1=approx1, approximant2=approx2, cond_num=1e25)

# net_true = gwnet.get_network_response(inj_params=inj_params, f_max=f_high, approximant=approx1, network_key=net_key, calc_detector_responses=False)

2023-11-20 12:49:21,134 - Network - INFO : Polarizations calculated.
2023-11-20 12:49:21,135 - Network - INFO : Calculate numeric derivatives of polarizations.
2023-11-20 12:49:21,143 - Network - INFO : Polarizations calculated.
2023-11-20 12:49:22,413 - Network - INFO : Numeric derivatives of polarizations calculated.
2023-11-20 12:49:22,488 - Network - INFO : PSDs loaded.
2023-11-20 12:49:22,491 - Network - INFO : Antenna patterns and LPFs loaded.
2023-11-20 12:49:22,520 - Network - INFO : Detector responses calculated.
2023-11-20 12:49:22,521 - Network - INFO : Calculate numeric derivatives of detector responses.
2023-11-20 12:49:22,522 - Network - INFO :    CE-40_C
2023-11-20 12:49:23,416 - Network - INFO :    CE-20_S
2023-11-20 12:49:24,331 - Network - INFO :    ET_ET1
2023-11-20 12:49:25,326 - Network - INFO :    ET_ET2
2023-11-20 12:49:26,179 - Network - INFO :    ET_ET3
2023-11-20 12:49:27,391 - Network - INFO : Numeric derivatives of detector responses calculated.
2023-11-20 1

In [41]:
net_ap.hfc
net_ap.f

array([  5.    ,   5.0625,   5.125 , ..., 194.8125, 194.875 , 194.9375])

In [51]:
step=1e-9
method='central' 
order=2
n=1

for det in net_ap.detectors:
    det.calc_det_responses_derivs_num(net_ap.inj_params, net_ap.deriv_variables, net_ap.wf, net_ap.deriv_symbs_string, net_ap.conv_cos, net_ap.conv_log, net_ap.use_rot, step, method, order, n, user_locs=None)
print('Numeric derivatives of detector responses calculated.')


Numeric derivatives of detector responses calculated.


In [54]:
from gwbench import wf_derivatives_num as wdn
wdn.part_deriv(net_ap.detectors[0].del_hf['del_Mc_hf'], net_ap.f, net_ap.deriv_variables, pl_cr=0, compl=None, step=1e-9, method='central', order=2, n=1)

TypeError: 'numpy.ndarray' object is not callable